<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo final de master</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Estado de la computación cuántica en el aprendizaje por refuerzo y cómo aplicarla en DQN y Reinforce con Línea Base

## QVC aplicado a DQN en entorno CartPole



Se importan las librerías necesarias para ejecutar el Jupyter Notebook

In [1]:
!pip uninstall qiskit-ibm-provider

In [2]:
# Importamos timeit para tomar mediciones de tiempo de ejecución
import timeit

# Importamos codecarbon para tomar medidas de huella de carbono
from codecarbon import EmissionsTracker

In [3]:
import pennylane as qml
import torch as torch
import torch.nn as nn
from torch.nn.parameter import Parameter

In [4]:
import sys
# Para importar nuestras librerías, que están en otro directorio
sys.path.append("../librerias")

# Experience Replay Buffer para DQN
import experienceReplayBuffer as erb

import Model

# Agentes
import Agent

In [5]:
# Se importa la librería gymnasium para utilizar los entornos CartPole y Acrobot
import gym as gym

In [6]:
def encode(n_qubits, inputs):
    for wire in range(n_qubits):
        qml.RX(inputs[wire], wires=wire)

def layer(n_qubits, y_weight, z_weight):
    for wire, y_weight in enumerate(y_weight):
        qml.RY(y_weight, wires=wire)
    for wire, z_weight in enumerate(z_weight):
        qml.RZ(z_weight, wires=wire)
    for wire in range(n_qubits):
        qml.CZ(wires=[wire, (wire + 1) % n_qubits])


def measure_cartpole(n_qubits):
    return [
        qml.expval(qml.PauliZ(0) @ qml.PauliZ(1)),
        qml.expval(qml.PauliZ(2) @ qml.PauliZ(3))
    ]


def get_model(n_qubits, n_layers, environment, quantum_device):
    dev = qml.device(quantum_device, wires=n_qubits, backend="ibm_nairobi", ibmqx_token="37e367fa79da3b20decc60f1160d6c8a4ecda602b6ab86a3d2f01a293b5e177498f692b0845cfd85852067fc3e06e391901bb976d318b7eb1e00e1a4e3a03ba0")
    shapes = {
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit_cartpole(inputs, y_weights, z_weights):
        for layer_idx in range(n_layers):
#            if (layer_idx == 0):
            encode(n_qubits, inputs)
            layer(n_qubits, y_weights[layer_idx], z_weights[layer_idx])
        return measure_cartpole(n_qubits)

    model = qml.qnn.TorchLayer(circuit_cartpole, shapes)

    return model

class QuantumNet(nn.Module):
    def __init__(self, n_layers, n_qubits, n_actions, environment, quantum_device):
        super(QuantumNet, self).__init__()
        self.n_qubits = n_qubits
        self.n_actions = n_actions
        self.environment = environment
#        if self.environment == 0:  # Cartpole
        self.w_input = Parameter(torch.Tensor(self.n_qubits))
        nn.init.normal_(self.w_input, mean=0.)
        self.w_output = Parameter(torch.Tensor(self.n_actions))
        nn.init.normal_(self.w_output, mean=90.)
        self.q_layers = get_model(n_qubits=self.n_qubits,
                                  n_layers=n_layers,
                                  environment=self.environment, quantum_device=quantum_device)

    def forward(self, inputs):
        inputs = torch.atan(inputs)

        inputs = inputs * self.w_input

        outputs = self.q_layers(inputs)
        outputs = (1 + outputs) / 2


#        if self.environment == 0:  # Cartpole
        outputs = outputs * self.w_output
        return outputs

Se crea el entorno, en este caso CartPole

In [7]:
env = gym.envs.make("CartPole-v0")

/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:592: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Se definen los hiperparámetros que se utilizarán en el modelo de aprendizaje por refuerzo

In [8]:
# Hiperparámetros del modelo de aprendizaje por refuerzo
lr = 0.01            #Velocidad de aprendizaje
lr_input=0.01
lr_output=0.01
MEMORY_SIZE = 32  #Máxima capacidad del buffer
MAX_EPISODES = 2   #Número máximo de episodios (el agente debe aprender antes de llegar a este valor)
EPSILON = 1           #Valor inicial de epsilon
EPSILON_DECAY = .99   #Decaimiento de epsilon
GAMMA = 0.99          #Valor gamma de la ecuación de Bellman
BATCH_SIZE = 8       #Conjunto a coger del buffer para la red neuronal
BURN_IN = 16        #Número de episodios iniciales usados para rellenar el buffer antes de entrenar
DNN_UPD = 1          #Frecuencia de actualización de la red neuronal
DNN_SYNC = 1     #Frecuencia de sincronización de pesos entre la red neuronal y la red objetivo

Se definen los hiperparámetros del circuito QVC

In [9]:
# Hiperparámetros del QVC
n_layers = 5    # número de capas del Ansatz
n_qubits = 4    # qubits definidos, en este caso son 4 al disponer de 4 variables de observación
n_actions = 2   # número de acciones, se definen 2 para Cartpole
environment = 0 # entorno Cartpole
quantum_device = "qiskit.ibmq" # dispositivo cuántico, se define un simulador cuántico

Se crea el buffer de repetición de experiencias

In [10]:
buffer = erb.experienceReplayBuffer(memory_size=MEMORY_SIZE, burn_in=BURN_IN)   # buffer experience replay

Se define el circuito cuántico que sustituye a la red neuronal

In [11]:
net = QuantumNet(n_layers, n_qubits, n_actions, environment, quantum_device)

/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/pennylane_qiskit/__init__.py:19: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  from .ibmq import IBMQDevice
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/pennylane_qiskit/ibmq.py:115: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  active_account = IBMQ.active_account()


Se crea la red objetivo también como un circuito cuántico

In [12]:
target_network = QuantumNet(n_layers, n_qubits, n_actions, environment, quantum_device)

Se crea el modelo DQN

In [13]:
dqn = Model.DQN(env, net, learning_rate=lr, lr_input=lr_input, lr_output=lr_output)

Se crea el modelo DQN objetivo

In [14]:
dqn_target = Model.DQN(env, target_network, learning_rate=lr, lr_input=lr_input, lr_output=lr_output)

Se crea el agente, se le pasa como parámetro los dos modelos inicializados que contienen un circuito cuántico

In [15]:
agent = Agent.DQNAgent(env, dqn, dqn_target, buffer, EPSILON, EPSILON_DECAY, BATCH_SIZE)

Se prepara el entorno para iniciar las mediciones de CO2

In [16]:
# Se define la carpeta de salida de los archivos de emisiones
output_dir = './emisiones/emisiones_QVC_DQN_CartPole'

# Se crea una instancia de EmissionsTracker y se configura la carpeta de salida
tracker = EmissionsTracker(output_dir=output_dir, log_level = "critical")

# Se inicializa el seguimiento de las emisiones
tracker.start()

Iniciamos el cronómetro para medir el tiempo del proceso

In [17]:
tiempo_inicio = timeit.default_timer()

Se inicia el entrenamiento del agente

In [ ]:
agent.train(gamma=GAMMA, max_episodes=MAX_EPISODES,
              batch_size=BATCH_SIZE, dnn_update_frequency=DNN_UPD, dnn_sync_frequency=DNN_SYNC)

Rellenando el buffer de repetición...
Entrenando...


/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/pennylane_qiskit/qiskit_device.py:462: UserWarning: ibmqx_token is not a recognized runtime option and may be ignored by the backend.
  self._current_job = self.backend.run(compiled_circuits, shots=self.shots, **self.run_args)
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/websocket/_http.py:210: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = ssl.SSLContext(sslopt.get('ssl_version', ssl.PROTOCOL_SSLv23))
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/websocket/_app.py:308: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread.setDaemon(True)
/Users/marcoemiliorodriguezserrano/anaconda3/lib/python3.10/site-packages/websocket/_http.py:210: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  context = ssl.SSLContext(sslopt.get('ssl_version', ssl.PROTOCOL_SSLv23))
/Users/marcoemiliorodriguezserrano/anaconda3/lib/

Se calcula el tiempo de ejecución, lo que ha tardado el agente en entrenar

In [ ]:
tiempo_ejecucion = round(timeit.default_timer() - tiempo_inicio, 0)
print("Tiempo ejecución entrenamiento: " + str(int(tiempo_ejecucion/3600))
      + " horas, " + str(int((tiempo_ejecucion % 3600)/60)) + " minutos y "
      + str(int((tiempo_ejecucion % 3600)%60)) + " segundos")


Se calculan las emisiones que han podido implicar el proceso

In [ ]:
emisiones = tracker.stop()
print("Emisiones de CO2 (Kg)" + str(emisiones))

Se procede a graficar los resultados

In [ ]:
agent.plot_rewards()
agent.plot_loss()

In [ ]:
import numpy as np
np.savetxt('./resultados_ejecuciones/T2_QVC_DQN_CartPole_param2_b_001_cuantico_real_tr.txt', agent.training_rewards)
np.savetxt('./resultados_ejecuciones/T2_QVC_DQN_CartPole_param2_b_001_mean_cuantico_real_tr.txt', agent.mean_training_rewards)
np.savetxt('./resultados_ejecuciones/T2_QVC_DQN_CartPole_param2_b_001_loss_cuantico_real.txt', agent.training_loss)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plt.plot(agent.training_rewards, label='Recompensas')
plt.plot(agent.mean_training_rewards, label='Recompensas medias')
plt.axhline(agent.reward_threshold, color='r', label="Límite recompensa")
plt.xlabel('Episodios')
plt.ylabel('Recompensas')
plt.legend(loc="upper left")
plt.show()


In [ ]:
print(agent.training_rewards)